In [1]:
# Core 
import datetime
import os
import glob
# from IPython.display import HTML, Image, display
import tempfile
import shutil
import sys
import pickle
import warnings

# Analysis 
import xarray as xr
import numpy as np
import pandas as pd

# Plotting 
# import cartopy.crs as ccrs
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
# import cartopy.feature as cfeature
# import matplotlib.pyplot as plt
# from matplotlib import rcParams
# import matplotlib.ticker as mticker
# from matplotlib.animation import FuncAnimation
# import matplotlib.colors as colors

# Debugging 
import pdb, traceback
%load_ext line_profiler

# Radar Tools
import pyart
import tint
from tint import animate
# from tint.visualization import embed_mp4_as_gif


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [ ]:
# def test():
# Store filenames for an example day of data
base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

filenames = sorted(glob.glob(base + '2017/201612*/*.nc') + glob.glob(base + '2017/20170[1-2]*/*.nc'))
# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') 
         for fn in filenames)

# Define settings for tracking
settings = {
    'MIN_SIZE' : [15, 15, 15, 15], # This probably size in pixel units or km. 
    'FIELD_THRESH' : [47, 40, 35, 30],
    'ISO_THRESH' : [4, 4, 4, 4],
    'GS_ALT' : 3000,
    'LEVELS' : np.array(
        [[0, 2000], 
         [2000, 4000],
         [4000, 6000],
         [6000, 8000]]
    ),
    'TRACK_INTERVAL' : 1,
}

# Calculate high and low level tracks
tracks_obj  = tint.Cell_tracks()

for parameter in ['MIN_SIZE', 'FIELD_THRESH', 'GS_ALT', 'LEVELS', 
                  'TRACK_INTERVAL', 'ISO_THRESH'
                 ]:
    tracks_obj.params[parameter] = settings[parameter]

# Calculate tracks
tracks_obj.get_tracks(grids)

out_file_name = '/g/data/w40/esh563/CPOL_analysis/tracks_obj_{}.pkl'.format(dt)

with open(out_file_name, 'wb') as f:
    pickle.dump(tracks_obj, f)
    
# if __name__ == '__main__':
#     try:
#         test()
#     except:
#         extype, value, tb = sys.exc_info()
#         traceback.print_exc()
#         pdb.post_mortem(tb)

In [16]:
with open(out_file_name, 'rb') as f:
    tracks_obj = pickle.load(f)

In [17]:
lon_min = 130.40
lon_max = 131.69

lat_max = -11.61
lat_min = -12.87

base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

# Note we only need grid generators for the grids that actually contain
# the system!
# filenames = sorted(glob.glob(base + '2017/20170101/*_1[6-9]*.nc'))

# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity')
         for fn in filenames)

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")    

base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
out_path = base_path + dt + '_tint_CPOL_anim'

animate(tracks_obj, grids, out_path,
        start_datetime = np.datetime64('2017-01-03'),
        end_datetime = np.datetime64('2017-01-04'),
        lat_lines=np.arange(lat_min, lat_max, .1),
        lon_lines=np.arange(lon_min, lon_max, .1),
        tracers=True, keep_frames=True, dpi=100)
    
# if __name__ == '__main__':
#     try:
#         test()
#     except:
#         extype, value, tb = sys.exc_info()
#         traceback.print_exc()
#         pdb.post_mortem(tb)

Animating from  2017-01-03  to  2017-01-04 .
Creating GIF - may take a few minutes.       


In [4]:
# Let's find the cells that were tracked for the most frames
tracks_obj.system_tracks.groupby(level='uid').size().sort_values(ascending=False)[:10]
# tracks_obj.system_tracks['n_cores'][tracks_obj.system_tracks['n_cores'] == 5]

uid
1677    27
490     26
1324    22
312     22
526     21
3       21
538     21
66      20
156     20
505     20
dtype: int64

In [42]:
tracks_obj.system_tracks.xs('0', level='uid', drop_level=False)

,,,grid_x,grid_y,lon,lat,u,v,n_cores,x_vert_disp,y_vert_disp,tilt_mag,vel_dir,tilt_dir,sys_rel_tilt_dir
scan,time,uid,,,,,,,,,,,,,
31,2016-12-01 05:10:02,0,47.174,28.87,130.8325,-12.6175,NaN,NaN,1,3.545,0.597,3.594918,NaN,9.559269,NaN


In [38]:
tracks_obj.tracks.query('time' < np.datetime64("2016-12-05"))

TypeError: '>' not supported between instances of 'numpy.ndarray' and 'str'

In [77]:
time_ind = tracks_obj.tracks.index.get_level_values('time')
cond = ((time_ind > np.datetime64('2016-12-05T04:00')) & (time_ind < np.datetime64('2016-12-05T12:20')))
tracks_obj.tracks[cond]

grid_x   grid_y       lon      lat  \
scan time                level uid                                       
610  2016-12-05 05:50:02 0     41   26.381  138.571  130.6405 -11.6281   
                         1     41   24.897  138.359  130.6313 -11.6370   
                         2     41   24.929  138.429  130.6313 -11.6370   
                         3     41   26.333  138.143  130.6404 -11.6371   
611  2016-12-05 06:00:07 0     41   23.968  139.000  130.6221 -11.6280   
                         1     41   23.575  138.750  130.6221 -11.6280   
                         2     41   23.342  138.474  130.6129 -11.6370   
                         3     41   23.000  138.286  130.6129 -11.6370   
647  2016-12-05 12:00:06 0     42   51.174   27.087  130.8693 -12.6355   
                         1     42   50.421   27.158  130.8601 -12.6355   
                         2     42   50.464   26.571  130.8601 -12.6355   
                         3     42   49.706   26.412  130.8601 -12.6445   
648  2016-12-05 12:10:02 0     42   48.966   28.310  130.8509 -12.6265   
                         1     42   48.196   28.471  130.8417 -12.6265   
                         2     42   48.000   26.528  130.8417 -12.6355   
                         3     42   47.774   26.484  130.8417 -12.6445   

                                    proj_area   vol        max  max_alt  \
scan time                level uid                                        
610  2016-12-05 05:50:02 0     41        21.0  30.5  56.224136      1.5   
                         1     41        39.0  66.0  56.577808      1.5   
                         2     41        42.0  63.5  53.619381      1.5   
                         3     41        21.0  28.0  36.757553      1.0   
611  2016-12-05 06:00:07 0     41        31.0  44.0  57.022305      1.5   
                         1     41        40.0  73.5  56.964188      1.5   
                         2     41        38.0  54.0  50.993759      1.5   
                         3     41        21.0  27.5  34.598061      1.5   
647  2016-12-05 12:00:06 0     42        23.0  33.5  56.540535      1.5   
                         1     42        38.0  64.5  58.788422      1.5   
                         2     42        28.0  37.0  50.256779      1.5   
                         3     42        17.0  25.5  43.152950      1.5   
648  2016-12-05 12:10:02 0     42        29.0  42.0  56.119705      1.5   
                         1     42        51.0  81.5  56.224869      1.5   
                         2     42        36.0  55.5  51.640415      1.5   
                         3     42        31.0  46.5  50.126694      1.5   

                                    isolated  n_cores         u         v  \
scan time                level uid                                          
610  2016-12-05 05:50:02 0     41       True        1       NaN       NaN   
                         1     41       True        1       NaN       NaN   
                         2     41       True        1       NaN       NaN   
                         3     41       True        1       NaN       NaN   
611  2016-12-05 06:00:07 0     41       True        1 -4.021667  0.715000   
                         1     41       True        1 -2.203333  0.651667   
                         2     41       True        1 -2.645000  0.075000   
                         3     41       True        1 -5.555000  0.238333   
647  2016-12-05 12:00:06 0     42      False        1       NaN       NaN   
                         1     42      False        1       NaN       NaN   
                         2     42       True        1       NaN       NaN   
                         3     42       True        1       NaN       NaN   
648  2016-12-05 12:10:02 0     42      False        1 -3.680000  2.038333   
                         1     42       True        1 -3.708333  2.188333   
                         2     42       True        1 -4.106667 -0.071667   
                         3     42       Tru

In [83]:
time_ind = tracks_obj.tracks.index.get_level_values('time')
cond = ((time_ind == np.datetime64('2016-12-05 12:00:06')))
tracks_obj.tracks[cond]

grid_x  grid_y       lon      lat  \
scan time                level uid                                      
647  2016-12-05 12:00:06 0     42   51.174  27.087  130.8693 -12.6355   
                         1     42   50.421  27.158  130.8601 -12.6355   
                         2     42   50.464  26.571  130.8601 -12.6355   
                         3     42   49.706  26.412  130.8601 -12.6445   

                                    proj_area   vol        max  max_alt  \
scan time                level uid                                        
647  2016-12-05 12:00:06 0     42        23.0  33.5  56.540535      1.5   
                         1     42        38.0  64.5  58.788422      1.5   
                         2     42        28.0  37.0  50.256779      1.5   
                         3     42        17.0  25.5  43.152950      1.5   

                                    isolated  n_cores   u   v  x_vert_disp  \
scan time                level uid                                           
647  2016-12-05 12:00:06 0     42      False        1 NaN NaN          NaN   
                         1     42      False        1 NaN NaN       -0.753   
                         2     42       True        1 NaN NaN        0.043   
                         3     42       True        1 NaN NaN       -0.758   

                                    y_vert_disp  
scan time                level uid               
647  2016-12-05 12:00:06 0     42           NaN  
                         1     42         0.071  
                         2     42        -0.587  
                         3     42        -0.159

In [61]:
tracks_obj.tracks.index.get_level_values('time')

DatetimeIndex(['2016-12-01 05:10:02', '2016-12-01 05:10:02',
               '2016-12-01 05:10:02', '2016-12-01 05:10:02',
               '2016-12-01 05:10:02', '2016-12-01 05:10:02',
               '2016-12-01 05:10:02', '2016-12-01 05:10:02',
               '2016-12-01 05:20:02', '2016-12-01 05:20:02',
               ...
               '2017-01-31 19:20:01', '2017-01-31 19:20:01',
               '2017-01-31 19:30:06', '2017-01-31 19:30:06',
               '2017-01-31 19:30:06', '2017-01-31 19:30:06',
               '2017-01-31 19:40:02', '2017-01-31 19:40:02',
               '2017-01-31 19:40:02', '2017-01-31 19:40:02'],
              dtype='datetime64[ns]', name='time', length=24608, freq=None)

In [84]:
list(enumerate([1,2]))

[(0, 1), (1, 2)]

In [3]:
base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

# Note we only need grid generators for the grids that actually contain
# the system!
filenames = sorted(glob.glob(base + '2017/20170129/*_1[6-9]*') + 
                    glob.glob(base + '2017/20170129/*_2[0-4]??_*.nc'))

grids = (pyart.io.read_grid(fn) for fn in filenames)  # refresh grid generator

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")

base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
out_path = base_path + dt + '_tint_CPOL_lagrangian'
animate(tracks_obj, grids, out_path, style='lagrangian', uid='1677', alt=3000,
        keep_frames=True, dpi=100) 

Animating 27 frames
Object not yet initiated at 2017-01-29 16:00:01.000.
Moving to next grid.
Object not yet initiated at 2017-01-29 16:10:01.000.
Moving to next grid.
Object not yet initiated at 2017-01-29 16:20:01.000.
Moving to next grid.
Object died at2017-01-29 20:50:02.0:02.000    
Ending loop.
Creating GIF - may take a few minutes.       


In [ ]:
cell.iloc[0].time == np.numpy64(grid.metadata['start_time'])